In [1]:
# ==============================================
# 1. 라이브러리 & 하드웨어 설정
# ==============================================
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# ==============================================
# 2. CIFAR‑10 데이터셋 & DataLoader
# ==============================================
#   - 학습용: 랜덤 크롭 + 좌우반전 + 정규화
#   - 검증용: 정규화만
# ==============================================

mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_ds = datasets.CIFAR10(root="./data", train=True,  download=True, transform=train_tf)
test_ds = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


In [6]:
# ==============================================
# 3. 사전학습 모델 로드 & 헤드(FC) 교체
# ==============================================
feature_extract = True

weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)

for n, _ in model.named_children():
    print(n)


conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [3]:
if feature_extract:
    for p in model.parameters():
        p.requires_grad = False

for name, p in model.named_parameters():
    if "layer4" in name:
        p.requires_grad = True


conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)        # CIFAR‑10 → 10‑class

model = model.to(device)


In [ ]:
# ==============================================
# 4. 학습 설정
# ==============================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)

In [4]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return (preds == labels).float().mean().item()


In [5]:
# ==============================================
# 5. 학습 & 검증 루프
# ==============================================
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    # ---------- Train ----------
    model.train()
    running_loss, running_acc = 0.0, 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_acc  += accuracy(outputs, labels) * images.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_acc  = running_acc  / len(train_loader.dataset)

    # ---------- Eval ----------
    model.eval()
    test_loss, test_acc = 0.0, 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss    = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            test_acc  += accuracy(outputs, labels) * images.size(0)

    test_loss /= len(test_loader.dataset)
    test_acc  /= len(test_loader.dataset)

    print(f"[Epoch {epoch}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}% "
          f"|| Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%")


[Epoch 1/50] Train Loss: 2.0676 | Train Acc: 25.91% || Test Loss: 1.9429 | Test Acc: 32.01%
[Epoch 2/50] Train Loss: 1.8365 | Train Acc: 35.00% || Test Loss: 1.8262 | Test Acc: 36.23%
[Epoch 3/50] Train Loss: 1.7805 | Train Acc: 37.21% || Test Loss: 1.7855 | Test Acc: 37.56%
[Epoch 4/50] Train Loss: 1.7478 | Train Acc: 38.44% || Test Loss: 1.7568 | Test Acc: 39.13%
[Epoch 5/50] Train Loss: 1.7288 | Train Acc: 38.90% || Test Loss: 1.7343 | Test Acc: 39.60%
[Epoch 6/50] Train Loss: 1.7169 | Train Acc: 39.50% || Test Loss: 1.7379 | Test Acc: 39.81%
[Epoch 7/50] Train Loss: 1.7087 | Train Acc: 39.74% || Test Loss: 1.7303 | Test Acc: 39.95%
[Epoch 8/50] Train Loss: 1.7055 | Train Acc: 40.11% || Test Loss: 1.7302 | Test Acc: 39.97%
[Epoch 9/50] Train Loss: 1.6972 | Train Acc: 40.43% || Test Loss: 1.7464 | Test Acc: 39.84%
[Epoch 10/50] Train Loss: 1.6882 | Train Acc: 40.56% || Test Loss: 1.7264 | Test Acc: 40.68%
[Epoch 11/50] Train Loss: 1.6842 | Train Acc: 40.64% || Test Loss: 1.7137 | Tes